# Lecture 2.1

## Structural induction on trees

# Lecture 2.2 - Streams

Streams are defined from a constant `Stream.empty` and a constructor `Stream.cons`.

In [1]:
val xs = Stream.cons(1, Stream.cons(2, Stream.empty))

xs: Stream.Cons[Int] = Stream(1, 2)

In [2]:
(1 to 10).toStream

res1: Stream[Int] = Stream(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [6]:
// Stream range
def streamRange(lo: Int, hi: Int): Stream[Int] = {
    if (lo >= hi) Stream.empty
    else Stream.cons(lo, streamRange(lo + 1, hi))
}

// Compare to list version - structure is isomorphic but behaviour is quite different
def listRange(lo: Int, hi: Int): List[Int] = {
    if (lo >= hi) Nil
    else lo :: listRange(lo + 1, hi)
}


defined function streamRange
defined function listRange

The `#::` operator produces a stream.

Example of stream implementation. Note that the only difference with the List implementation is the call by name
argument for `tl` in the `cons` method definition.

`tl: => Stream[T]`

```scala
object Stream {
    def cons[T](hd: T, tl: => Stream[T]) = new Stream[T] {
        def isEmpty = false
        def head = hd
        def tail = tl
    }
    
    val empty = new Stream[Nothing] {
        def isEmpty = true
        def head = throw new NoSuchElementException("empty.head")
        def tail = throw new NoSuchElementException("empty.tail")
    }
}
```

# Lecture 2.3 - Lazy Evaluation

In [8]:
def expr = {
    val x = {print("x"); 1}
    lazy val y = {print("y"); 2}
    def z = {print("z"); 3}
    
    z + y + x + z + y + x
}

expr

xzyz

defined function expr
res7_1: Int = 12

More efficient version with lazy evaluated version of `tail`.

```scala
object Stream {
    def cons[T](hd: T, tl: => Stream[T]) = new Stream[T] {
        def isEmpty = false
        def head = hd
        lazy val tail = tl
    }
    
    val empty = new Stream[Nothing] {
        def isEmpty = true
        def head = throw new NoSuchElementException("empty.head")
        def tail = throw new NoSuchElementException("empty.tail")
    }
}
```

# Lecture 2.4 - Computing with Infinite Sequences

In [9]:
def from(n: Int): Stream[Int] =
    n #:: from(n + 1)

defined function from

In [10]:
from(0) take 3

res9: Stream[Int] = Stream(0, 1, 2)

In [11]:
from(2) take 4

res10: Stream[Int] = Stream(2, 3, 4, 5)

In [14]:
val nats = from(0)

nats: Stream[Int] = Stream(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
...

In [16]:
val m4s = nats map (_ * 4)

m4s: Stream[Int] = Stream(
  0,
  4,
  8,
  12,
  16,
  20,
  24,
  28,
  32,
  36,
  40,
  44,
  48,
  52,
  56,
  60,
  64,
  68,
  72,
  76,
  80,
  84,
  88,
  92,
  96,
  100,
  104,
  108,
  112,
  116,
  120,
  124,
  128,
  132,
  136,
  140,
  144,
  148,
...

## The Sieve of Erastothenes

In [17]:
def sieve(s: Stream[Int]): Stream[Int] = {
    s.head #:: sieve(s.tail filter(_ % s.head != 0))
}

defined function sieve

In [21]:
val primes = sieve(from(2))

primes: Stream[Int] = Stream(
  2,
  3,
  5,
  7,
  11,
  13,
  17,
  19,
  23,
  29,
  31,
  37,
  41,
  43,
  47,
  53,
  59,
  61,
  67,
  71,
  73,
  79,
  83,
  89,
  97,
  101,
  103,
  107,
  109,
  113,
  127,
  131,
  137,
  139,
  149,
  151,
  157,
  163,
...

In [22]:
def sqrtStream(x: Double): Stream[Double] = {
    def improve(guess: Double) = (guess + x / guess) / 2
    lazy val guesses: Stream[Double] = 1 #:: (guesses map improve)
    guesses
}

defined function sqrtStream

In [27]:
sqrtStream(4) take 7

res26: Stream[Double] = Stream(
  1.0,
  2.5,
  2.05,
  2.000609756097561,
  2.0000000929222947,
  2.000000000000002,
  2.0
)

# Lecture 2.5 - The Water Pouring Problem

In [59]:
class Pouring(capacity: Vector[Int]) {
    // States
    type State = Vector[Int]
    val initialState = capacity map (x => 0)
    
    // Moves
    trait Move {
        def change(state: State): State
    }
    case class Empty(glass: Int) extends Move {
        def change(state: State): State = {
            state updated (glass, 0)
        }
    }
    case class Fill(glass: Int) extends Move {
        def change(state: State): State = {
            state updated (glass, capacity(glass))
        }
    }
    case class Pour(from: Int, to: Int) extends Move {
        def change(state: State): State = {
            val amount = state(from) min (capacity(to) - state(to))
            state updated (from, state(from) - amount) updated (to, state(to) + amount)
        }
    }
    
    val glasses = 0 until capacity.length
    
    val moves = 
        (for (g <- glasses) yield Empty(g)) ++
        (for (g <- glasses) yield Fill(g)) ++
        (for (from <- glasses; to <- glasses if from != to) yield Pour(from, to))
    
    // Paths
    class Path(history: List[Move], val endState: State) {
        // Avoid recomputing endState 
        // 
        // def endState: State = trackState(history)
        // private def trackState(xs: List[Move]): State = xs match {
        //     case Nil => initialState
        //     case move :: xs1 =>  move change trackState(xs1)
        // }
        
        def extend(move: Move) = new Path(move :: history, move change endState)
        
        override def toString = (history.reverse mkString " ") + "->" + endState
    }
    
    val initialPath = new Path(Nil, initialState)
    
    def from(paths: Set[Path], explored: Set[State]): Stream[Set[Path]] = {
        if (paths.isEmpty) Stream.empty
        else {
            val more = for {
                path <- paths
                next <- moves map path.extend
                if !(explored contains next.endState)
            } yield next
            paths #:: from(more, explored ++ (more map(_.endState)))
        }
    }
    
    val pathSets = from(Set(initialPath), Set(initialState))
    
    def solutions(target: Int): Stream[Path] = {
        for {
            paths <- pathSets
            path <- paths
            if ((path endState) contains target)
        } yield path
    }
}

defined class Pouring

In [60]:
val x = new Pouring(Vector(4, 9))
x.solutions(6)

x: Pouring = ammonite.$sess.cmd58$Helper$Pouring@48d878e3
res59_1: Stream[x.Path] = Stream(
  Fill(1) Pour(1,0) Empty(0) Pour(1,0) Empty(0) Pour(1,0) Fill(1) Pour(1,0)->Vector(4, 6),
  Fill(1) Pour(1,0) Empty(0) Pour(1,0) Empty(0) Pour(1,0) Fill(1) Pour(1,0) Empty(0)->Vector(0, 6)
)

## `foldRight` version
```scala
class Path(history: List[Move]) {
    def endState: State = (history foldRight initialState)(_ change _)
}
```

In [42]:
val x = new Pouring(((1 to 10).toVector))
x.initialState

x: Pouring = ammonite.$sess.cmd40$Helper$Pouring@7b3dce6e
res41_1: Vector[Int] = Vector(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [44]:
val problem = new Pouring(Vector(4, 7))
problem.moves

problem: Pouring = ammonite.$sess.cmd40$Helper$Pouring@59409f67
res43_1: collection.immutable.IndexedSeq[Product with Serializable with problem.Move] = Vector(Empty(0), Empty(1), Fill(0), Fill(1), Pour(0, 1), Pour(1, 0))